In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
import time

# from mock.tasks import adiciona_carro}
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder \
           .appName('SparkByExamples') \
           .config("spark.jars", "/usr/share/java/mariadb-java-client.jar") \
           .getOrCreate()
sql = SQLContext(ss)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

session.execute("USE simulacao")

import json
params = json.load(open('./mock/parametros.json'))

In [19]:
a, b = 20, 50
query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao >= {a} AND tempo_da_simulacao <= {b} ALLOW FILTERING;"
r = list(session.execute(query))
if r != []:
    df = ss.createDataFrame(r)

In [6]:
a, b = 100, 500
query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao >= {a} AND tempo_da_simulacao <= {b} ALLOW FILTERING;"
r = list(session.execute(query))
if r != []:
    df2 = ss.createDataFrame(r)

In [20]:
df.count()

107

In [7]:
df2.count()

534

In [21]:
df.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|                29|ARG1N23|475.0|              89.1|
| BR-116|                29|CHI0G12|745.0|             791.0|
| BR-116|                29|PER1H23|385.0|254.75000000000006|
| BR-116|                24|ARG1N23|475.0|             71.85|
| BR-116|                24|CHI0G12|745.0|            812.75|
| BR-116|                24|PER1H23|385.0|198.00000000000006|
| BR-116|                44|ARG1N23|475.0|163.35000000000002|
| BR-116|                44|ARG9X01|745.0|             816.5|
| BR-116|                44|CHI0G12|745.0|            733.25|
| BR-116|                44|PER1H23|385.0| 447.5000000000002|
| BR-116|                42|ARG1N23|475.0|             151.5|
| BR-116|                42|ARG9X01|745.0|            834.15|
| BR-116|                42|CHI0G12|745.0|             740.3|
| BR-116

In [22]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, col

p = [[key]+list(params[key].values()) for key in params.keys()]
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))

Velocidades_Maximas = p.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
Aceleracoes_Maximas = p.select(F.col('rodovia'), 0.8*F.col("AceleracaoMaxima"))
Aceleracoes_Maximas = Aceleracoes_Maximas.withColumnRenamed("(AceleracaoMaxima * 0.8)", "AceleracaoMaxima")

collision_tolerance = 1
collision_tolerance_quad = 0.5
windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")

df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
df = df.withColumn("prev_tempo_da_simulacao", lag("tempo_da_simulacao", 1).over(windowSpec))
df = df.withColumn("vel_y", (col("pos_y") - col("prev_pos_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
df = df.withColumn("prev_vel_y", lag("vel_y", 1).over(windowSpec))
df = df.withColumn("acel_y", (col("vel_y") - col("prev_vel_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))

df = df.withColumn("posicao_prevista", col("pos_y") + col("vel_y") * (collision_tolerance) + col("acel_y") * collision_tolerance_quad)

window_spec_rf = Window.partitionBy("rodovia", "pos_x").orderBy('pos_y')
lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec_rf)
lead_column = lead(col("posicao_prevista")).over(window_spec_rf) - col("posicao_prevista")

# Add the lag column to the DataFrame
df = df.withColumn("Risco_Colisão", when(((lag_column < 0) & (col("rodovia") == lag(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lag(col("pos_x")).over(window_spec_rf)))| ((lead_column < 0) & (col("rodovia") == lead(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lead(col("pos_x")).over(window_spec_rf))), 1).otherwise(0))

df = df.join(Velocidades_Maximas,on='rodovia',how='left')
df = df.join(Aceleracoes_Maximas,on='rodovia',how='left')

df = df.withColumn('acima_vel',F.abs(col('vel_y'))>F.abs(col('VelocidadeMaxima')))
df = df.withColumn('acima_acel',F.abs(col('acel_y'))>F.abs(col('AceleracaoMaxima')))

df = df.withColumn("troca_faixa", col("pos_x") != lag("pos_x", 1).over(windowSpec))

# contador de trocas

df = df.withColumn('multado',((F.col('acima_vel') == 1) & (lag('acima_vel').over(windowSpec) == 0)))

windowSpec = Window.partitionBy("placa",'rodovia').orderBy('tempo_da_simulacao')

df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
df = df.withColumn("on_road", (((col("pos_y") > 0) & (col('pos_y') < 800)) & (col("prev_pos_y") > 0)).cast("int"))
df = df.withColumn('time_on_road',F.lit(0))
# df = df.withColumn("time_on_road", col("on_road") * (1 + lag("time_on_road", 1).over(windowSpec)))

# windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao").rowsBetween(Window.unboundedPreceding, 0)

# # df = df.withColumn("cumulative_sum", F.sum(col("on_road")).over(windowSpec))

# # # windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")

# df = df.withColumn("reset", when(col("time_on_road") != lag("time_on_road", 1).over(windowSpec), 1).otherwise(0))
# df = df.withColumn("group", F.sum("reset").over(windowSpec))
# df = df.withColumn("cumulative_sum", F.sum(col("placa")).over(windowSpec.partitionBy("group")))

In [23]:
dfp = df.toPandas()
dfp[dfp['placa'] == 'ARG1N23'].sort_values(by=['tempo_da_simulacao'], ascending=True)


,rodovia,tempo_da_simulacao,placa,pos_x,pos_y,prev_pos_y,prev_tempo_da_simulacao,vel_y,prev_vel_y,acel_y,posicao_prevista,Risco_Colisão,VelocidadeMaxima,AceleracaoMaxima,acima_vel,acima_acel,troca_faixa,multado,on_road,time_on_road
0,BR-116,20,ARG1N23,475.0,60.75,NaN,NaN,NaN,NaN,NaN,NaN,0,9,0.12,None,None,None,None,NaN,0
1,BR-116,21,ARG1N23,475.0,63.30,60.75,20.0,2.55,NaN,NaN,NaN,0,9,0.12,False,None,False,False,1.0,0
2,BR-116,22,ARG1N23,475.0,66.00,63.30,21.0,2.70,2.55,0.15,68.775,0,9,0.12,False,True,False,False,1.0,0
3,BR-116,23,ARG1N23,475.0,68.85,66.00,22.0,2.85,2.70,0.15,71.775,0,9,0.12,False,True,False,False,1.0,0
4,BR-116,24,ARG1N23,475.0,71.85,68.85,23.0,3.00,2.85,0.15,74.925,0,9,0.12,False,True,False,False,1.0,0
5,BR-116,25,ARG1N23,475.0,75.00,71.85,24.0,3.15,3.00,0.15,78.225,0,9,0.12,False,True,False,False,1.0,0
6,BR-116,26,ARG1N23,475.0,78.30,75.00,25.0,3.30,3.15,0.15,81.675,0,9,0.12,False,True,False,False,1.0,0
7,BR-116,27,ARG1N23,475.0,81.75,78.30,26.0,3.45,3.30,0.15,85.275,0,9,0.12,False,True,False,False,1.0,0
8,BR-116,28,ARG1N23,475.0,85.35,81.75,27.0,3.60,3.45,0.15,89.025,0,9,0.12,False,True,False,False,1.0,0
9,BR-116,29,ARG1N23,475.0,89.10,85.35,28.0,3.75,3.60,0.15,92.925,0,9,0.12,False,True,False,False,1.0,0


In [12]:
data = df.select('placa','pos_x','pos_y','acel_y','vel_y', 'tempo_da_simulacao',F.col('multado').cast('int'),'Risco_Colisão',F.col('troca_faixa'),'acima_vel')

In [10]:
# url = "jdbc:mariadb://host.docker.internal:3306/dashboard"

# data.write \
#   .format("jdbc") \
#   .option("driver","org.mariadb.jdbc.Driver") \
#   .option("url", url) \
#   .option("dbtable", "carros") \
#   .option("user", "root") \
#   .option("password", "secret") \
#   .save()


In [17]:
data.show()

+-------+-----+------+-------------------+------------------+------------------+-------+-------------+-----------+---------+
|  placa|pos_x| pos_y|             acel_y|             vel_y|tempo_da_simulacao|multado|Risco_Colisão|troca_faixa|acima_vel|
+-------+-----+------+-------------------+------------------+------------------+-------+-------------+-----------+---------+
|ARG1N23|475.0| 60.75|               null|              null|                20|   null|            0|       null|     null|
|ARG1N23|475.0|  63.3|               null| 2.549999999999997|                21|      0|            0|      false|    false|
|ARG1N23|475.0|  66.0|0.15000000000000568| 2.700000000000003|                22|      0|            0|      false|    false|
|ARG1N23|475.0| 68.85|0.14999999999999147|2.8499999999999943|                23|      0|            0|      false|    false|
|ARG1N23|475.0| 71.85|0.15000000000000568|               3.0|                24|      0|            0|      false|    false|


In [13]:
datap = data.toPandas()


In [20]:
import mysql.connector as database

connection = database.connect(
    host="host.docker.internal",
    port=3306,
    user="root",
    password="secret"
)

cursor = connection.cursor()
cursor.execute("USE dashboard;")


CREATE TABLE carros (
    placa varchar(10)  NOT NULL,
    pos_x float(6,3)  NOT NULL,
    pos_y float(6,3)  NOT NULL,
    aceleracao float(6,3)  NULL,
    velocidade float(6,3)  NULL,
    multas int  NULL,
    risco_colisao bool  NULL,
    direcao_perigosa bool  NULL,
    velocidade_acima bool  NULL,
    CONSTRAINT carros_pk PRIMARY KEY (placa)
);


In [52]:
str(list(datap.columns))

"['placa', 'pos_x', 'pos_y', 'acel_y', 'vel_y', 'multado', 'Risco_Colisão', 'troca_faixa', 'acima_vel']"

In [53]:
datap2 = datap.iloc[:1,:]

In [1]:
query = f""" INSERT IGNORE INTO carros (placa, pos_x, pos_y, aceleracao, velocidade, horario_registro, multas, risco_colisao, direcao_perigosa, velocidade_acima)

                VALUES {','.join([str(i) for i in list(datap.to_records(index=False))])};
                
                """

query = query.replace("\n", "")
query = query.replace("nan", "NULL")
query = query.replace("None", "NULL")

NameError: name 'datap' is not defined

In [55]:
query

" INSERT IGNORE INTO carros (placa, pos_x, pos_y, aceleracao, velocidade, multas, risco_colisao, direcao_perigosa, velocidade_acima)                VALUES ('BOL9D01', 835., 839.05, NULL, NULL, NULL, 0, NULL, NULL),('BOL9D01', 835., 817.9, NULL, -2.35, 0., 0, False, False),('BOL9D01', 835., 866., 50.45, 48.1, 1., 0, False, True),('BOL9D01', 835., 770.3, -47.975, -47.85, 0., 0, False, True),('BOL9D01', 835., 781., 6.1484375, 1.3375, 0., 0, False, False),('BRA2A34', 745., 835.4, NULL, NULL, NULL, 0, NULL, NULL),('BRA2A34', 745., 795.5, NULL, -13.3, NULL, 0, False, True),('BRA2A34', 745., 798.55, 7.4125, 1.525, 0., 0, False, False),('BRA2A34', 745., 854.15, 1.29027778, 9.26666667, 1., 0, False, True),('BRA6W78', 745., 811.8, NULL, NULL, NULL, 0, NULL, NULL),('BRA6W78', 745., 817.4, NULL, 2.8, 0., 0, False, False),('CHI6S78', 835., 817.25, NULL, NULL, NULL, 0, NULL, NULL),('CHI6S78', 835., 786.8, NULL, -15.225, NULL, 0, False, True),('CHI6S78', 835., 808.3, 36.725, 21.5, 0., 1, False, True)

In [56]:
cursor.execute(query)

In [57]:
connection.commit()

In [18]:
from sqlalchemy import create_engine


In [ ]:
pd.__version__

In [ ]:
connection = database.connect(
    host="host.docker.internal",
    port=3306,
    user="root",
    password="secret"
)
datap.to_sql("carros", con=connection, if_exists="append", index=False)

    placa varchar(10)  NOT NULL,
    pos_x float(6,3)  NOT NULL,
    pos_y float(6,3)  NOT NULL,
    aceleracao float(6,3)  NULL,
    velocidade float(6,3)  NULL,
    multas int  NULL,
    risco_colisao bool  NULL,
    direcao_perigosa bool  NULL,
    velocidade_acima bool  NULL,
    CONSTRAINT carros_pk PRIMARY KEY (placa)

In [ ]:
# # Exportar o pyspark dataframe para o mariadb

url = "jdbc:mariadb://host.docker.internal:3306/dashboard"

# data.write.jdbc(url=url, table="carros", mode="append", properties={"user": "root", "password": "secret"})


data.write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", url) \
  .option("dbtable", "carros") \
  .option("user", "root") \
  .option("password", "secret") \
  .save()


In [ ]:
# Exportar o pyspark dataframe para o mariadb

url = "jdbc:mysql://host.docker.internal:3306/dashboard"

data.write.jdbc(url=url, table="carros", mode="append", properties={"user": "root", "password": "secret", "driver": "org.mariadb.jdbc.Driver"})
